In [1]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np


C:\Users\SMest\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\SMest\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\SMest\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)
In C:\Users\SMest\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\SMest\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\SMest\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will b

In [2]:
import glob
from ipfx import feature_extractor
import os
import pyabf
from IPython.display import clear_output

In [3]:

abf_list_dir = glob.glob("*.abf")
abf_list = [pyabf.ABF(x) for x in abf_list_dir]
abf_list_name = [str(x) for x in abf_list_dir]
pairs = [c for c in zip(abf_list_name, abf_list)]
test=feature_extractor.SpikeFeatureExtractor()

In [7]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )
 
class abfanalysis(widgets.Box):
     
    def __init__(self):
        super().__init__()
        output = widgets.Output()
        
        plt.close('all') 
        with output:
            self.fig, self.abf_plot = plt.subplots(constrained_layout=True, figsize=(15, 7.5))
        self.abf_list = abf_list
        self.pairs = pairs
        self.abf = abf_list[0]
        self.fig.canvas.toolbar_position = 'bottom'
        self.lowerline = None
        self.upperline = None
        self.abf_plot.grid(True)
        #Starting vars
        self.sweep_list = [0,5,14]
        self.lowerlim = 0
        self.upperlim = 1
        self.maxinterval = 0.05
        self.minheight = 5
        self.minpeak = -10
        self.threshfrac = 5
        self.xlimit = [0, 2]
        self.alim = [0,2]
        self.dvdt = 20
        self.sweep_data = [None, None, None]
        self.sweepwise_spikes = [None, None, None]
        self.sweepwise_spikes_plot = [None, None, None]
        self.dvdt_line = None
        # define widgets
        self.dir = widgets.FileUpload(
                accept='.abf',  
                multiple=False 
            )
        self.abf_file = widgets.Dropdown(
            options=self.pairs,
            description='Number:',
        )
        self.sweeps = widgets.Text(
                value='0, 0',
                placeholder='0, 0',
                description='Sweeps:',
            continuous_update=False,
                disabled=False
            )
        xanalysis = widgets.FloatRangeSlider(
                value=[0, 2],
                min=0,
                max=self.abf.sweepX[-1],
                step=0.1,
                description='Analysis range:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='.1f',
            )
        xlim = widgets.FloatRangeSlider(
                value=[0, 2],
                min=0,
                max=self.abf.sweepX[-1],
                step=0.1,
                description='X limits:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='.1f',
            )
        dvdt = widgets.FloatText(
            value=20.0,
            description='dV/dT:',
            disabled=False
        )
        Maxinterval = widgets.FloatText(
            value=5,
            description='max interval tp (ms):',
            disabled=False
        )
        minheight = widgets.FloatText(
            value=5.0,
            description='min height (mV):',
            disabled=False
        )
        minpeak = widgets.FloatText(
            value=-10.0,
            description='min peak (mV):',
            disabled=False
        )
        threshfrac = widgets.FloatText(
            value=5,
            description='thres frac (%):',
            disabled=False
        )
        
 
        controls = widgets.VBox([
                                 self.abf_file,
            self.sweeps,
            xlim,
            xanalysis,
            dvdt,
            Maxinterval,
            minheight,
            minpeak,
            threshfrac
        ])
        controls.layout = make_box_layout()
        self.df_out = widgets.Output()
        out_box = widgets.VBox([output, self.df_out])
        output.layout = make_box_layout()
        self.df_out.layout = make_box_layout()
        
        
        
        # observe stuff
        self.dir.observe(self.update_abf_list, 'value')
        self.abf_file.observe(self.update_abf, 'value')
        xanalysis.observe(self.parse_analysis_lim, 'value')
        xlim.observe(self.update_xlim, 'value')
        dvdt.observe(self.update_dvdt, 'value')
        self.sweeps.observe(self.parse_sweeps, 'value')
        Maxinterval.observe(self.update_max_int, 'value')
        minheight.observe(self.update_min_height, 'value')
        minpeak.observe(self.update_min_peak, 'value')
        threshfrac.observe(self.update_threshfrac, 'value')
        self.plot_abf()
        self.update_dvdt_line
        self.run_analysis()
        self.plot_analysis()
        # add to children
        self.children = [controls, out_box]
     
        
    def update_lower_line(self, lwlim):
        self.lowerlim = lwlim
        if self.lowerline is not None:
            self.lowerline.set_xdata([self.lowerlim, self.lowerlim])
        else:
            plot = self.abf_plot
            self.lowerline = plot.axvline(self.lowerlim)
    def update_upper_line(self, uplim):
        self.upperlim = uplim
        if self.upperline is not None:
            self.upperline.set_xdata([self.upperlim, self.upperlim])
        else:
            plot = self.abf_plot
            self.upperline = plot.axvline(self.upperlim)
            
    def parse_analysis_lim(self, change):
            self.alim = change.new
            self.update_lower_line(self.alim[0])
            self.update_upper_line(self.alim[1])
            self.update_dvdt_line()
            self.run_analysis()
            self.plot_analysis()
        
    def update_dvdt(self, change):
        self.dvdt = change.new
        self.update_dvdt_line()
        self.run_analysis()
        self.plot_analysis()
    
        
    def update_dvdt_line(self):
        if self.dvdt_line is not None:
            self.dvdt_line.set_ydata([self.dvdt, self.dvdt])
        else:
            plot = self.dvdt_plot
            self.dvdt_line = plot.axhline(self.dvdt)
        
    def run_analysis(self):
        spikext = feature_extractor.SpikeFeatureExtractor(filter = 0, dv_cutoff=self.dvdt, start=self.alim[0], 
                                                          end=self.alim[1], max_interval=self.maxinterval,
                                                         min_height=self.minheight, min_peak=self.minpeak,
                                                         thresh_frac=self.threshfrac)
        for i,x in enumerate(self.sweep_list):
            self.abf.setSweep(x)
            dataT, dataV, dataI = self.abf.sweepX, self.abf.sweepY, self.abf.sweepC
            spikes_df = spikext.process(dataT, dataV, dataI)
            self.sweep_data[i] = spikes_df
        for i, x in enumerate(self.sweep_data):
            if x.empty == False:
                spikes_t = x['peak_t'].to_numpy()
                spikes_v = x['peak_v'].to_numpy()
                self.sweepwise_spikes[i] = np.vstack([spikes_t, spikes_v])
            else:
                self.sweepwise_spikes[i] = None
                try:
                    self.sweepwise_spikes_plot[i].remove()
                    self.sweepwise_spikes_plot[i] = None
                except:
                    self.sweepwise_spikes_plot[i]
        with self.df_out:
            clear_output()
            display(self.sweep_data[i])
            
    def plot_analysis(self):
        plot = self.abf_plot
        for i, x in enumerate(self.sweepwise_spikes):
            try:
                    self.sweepwise_spikes_plot[i].remove()
                    self.sweepwise_spikes_plot[i] = None
            except:
                    self.sweepwise_spikes_plot[i]
            if x is not None:
                self.sweepwise_spikes_plot[i] = plot.scatter(x[0,:],x[1,:])
    
    def update_abf(self, change):
        self.abf = change["new"]
        self.plot_abf()
        self.run_analysis()
        self.plot_analysis()
        
    def parse_sweeps(self, change):
        try:
            self.sweep_list = np.fromstring(change.new, dtype=int, sep=',')
        except:
            self.sweep_list = [0]
        self.sweep_data = [None for x in self.sweep_list]
        self.sweepwise_spikes = np.copy(self.sweep_data).tolist()
        self.sweepwise_spikes_plot = np.copy(self.sweep_data).tolist()
        
        self.plot_abf()
        self.run_analysis()
        self.plot_analysis()
        
    def plot_abf(self):
        """Draw line in plot"""
        self.fig.clear()
        self.upperline, self.lowerline = None, None
        self.abf_plot, self.dvdt_plot = self.fig.subplots(nrows=2, ncols=1, sharex=True)
        for x in self.sweep_list:
            #ax = self.fig.add_axes(figsize=(5, 3.5)sharex=True, sharey=True)
            self.abf.setSweep(x)
            self.abf_plot.plot(self.abf.sweepX, self.abf.sweepY)
            self.dvdt_plot.plot(self.abf.sweepX[:-1], (np.diff(self.abf.sweepY)/np.diff(self.abf.sweepX))/1000)
            self.update_dvdt_line()
        self.fig.gca().set_xlim(self.xlimit)
        
        self.fig.canvas.draw()
        self.update_lower_line(self.alim[0])
        self.update_upper_line(self.alim[1])
        
    def update_xlim(self, change):
        try:
            self.xlimit = change.new
            self.fig.gca().set_xlim(self.xlimit)
        except:
            return
    def update_abf_list(self, change):
        abf_dir = self.dir.value.keys()
        self.abf_list = []
        nan_str = 'nan'
        for x in abf_dir:
            with open("./temp.abf", "wb") as fp:
                fp.write(self.dir.value[x]['content'])
            temp_abf = pyabf.ABF("./temp.abf")
            os.remove("./temp.abf")
            self.abf_list.append(temp_abf)
        self.abf_list_name = [x for x in abf_dir]
        self.pairs = [c for c in zip(self.abf_list_name, self.abf_list)]

        self.abf_file.options = self.pairs
        
    
    def update_max_int(self, change):
        self.maxinterval = change.new/1000
        self.run_analysis()
        self.plot_analysis()
    
    def update_min_height(self, change):
        self.minheight = change.new
        self.run_analysis()
        self.plot_analysis()
        
    def update_min_peak(self, change):
        self.minpeak = change.new
        self.run_analysis()
        self.plot_analysis()
    
    def update_threshfrac(self, change):
        self.threshfrac = change.new
        self.run_analysis()
        self.plot_analysis()
         
abfanalysis()

AttributeError: 'abfanalysis' object has no attribute 'dir'